In [ ]:
dir_csv = '../input/rsna-intracranial-hemorrhage-detection'
dir_train_img = '../input/rsna-train-stage-1-images-png-224x/stage_1_train_png_224x'
dir_test_img = '../input/rsna-test-stage-1-images-png-224x/stage_1_test_png_224x'

In [ ]:
dir_csv = dir_dcm = '/home/jupyter/rsna/source_data'
dir_train_img = '/home/jupyter/rsna/rsna-train-stage-1-images-png-224x/stage_1_train_png_224x'
dir_test_img = '/home/jupyter/rsna/rsna-train-stage-1-images-png-224x/stage_1_test_png_224x'


In [16]:
import os
from os import path
import cv2
import glob
import pydicom
import numpy as np
# import pandas as pd
import torch
import torch.optim as optim
import sys
from tqdm import tqdm_notebook as tqdm
from matplotlib import pyplot as plt
import random
# from pathlib import Path
from sklearn.model_selection import train_test_split, KFold, GroupKFold

from fastai2.torch_basics      import *
from fastai2.data.all          import *
from fastai2.test              import *
from fastai2.medical.imaging   import *



In [17]:
np.random.seed(42)
random.seed(42)

In [2]:
saved = 'saved'
if not path.exists(saved):
    os.mkdir(saved)

In [3]:
dir_csv = dir_dcm = '/home/jupyter/rsna/source_data'
dir_train_img = '/home/jupyter/rsna/rsna-train-stage-1-images-png-224x/stage_1_train_png_224x'
dir_test_img = '/home/jupyter/rsna/rsna-train-stage-1-images-png-224x/stage_1_test_png_224x'


root_path = Path(dir_dcm)
path_trn = root_path/'stage_1_train_images'
fns_trn = path_trn.ls()

path_tst = root_path/'stage_1_test_images'
fns_tst = path_tst.ls()

In [ ]:
%time df_tst = pd.DataFrame.from_dicoms(fns_tst, px_summ=True, n_workers=8)
df_tst.to_feather('df_tst.fth')
df_tst.head()

In [ ]:
%time df_trn = pd.DataFrame.from_dicoms(fns_trn, px_summ=True, n_workers=8)
df_trn.to_feather('df_trn.fth')
df_trn.head()

In [4]:
df_trn = pd.read_feather('df_trn.fth')
df_trn.head()

,SOPInstanceUID,Modality,PatientID,StudyInstanceUID,SeriesInstanceUID,StudyID,ImagePositionPatient,ImageOrientationPatient,SamplesPerPixel,PhotometricInterpretation,...,MultiPixelSpacing,PixelSpacing1,img_min,img_max,img_mean,img_std,MultiWindowCenter,WindowCenter1,MultiWindowWidth,WindowWidth1
0,ID_231d901c1,CT,ID_b81a287f,ID_dd37ba3adb,ID_15dcd6057a,,-125.0,1.0,1,MONOCHROME2,...,1,0.488281,-1024,3263,171.462490,828.102464,NaN,NaN,NaN,NaN
1,ID_994bc0470,CT,ID_400facde,ID_c5277f0c63,ID_4ba12c2161,,-125.0,1.0,1,MONOCHROME2,...,1,0.488281,0,2507,430.418091,599.742963,1.0,47.0,1.0,80.0
2,ID_127689cce,CT,ID_42910d3d,ID_db93ade25b,ID_c4b4931314,,-125.0,1.0,1,MONOCHROME2,...,1,0.488281,-2000,2810,12.801376,1209.046168,NaN,NaN,NaN,NaN
3,ID_25457734a,CT,ID_329aafa7,ID_8dd6d32f3b,ID_116558f409,,-114.0,1.0,1,MONOCHROME2,...,1,0.445312,0,2647,566.557011,610.152845,1.0,36.0,1.0,80.0
4,ID_81c9aa125,CT,ID_6b544c3c,ID_2685c5d5c0,ID_f56d7bd0f9,,-115.0,1.0,1,MONOCHROME2,...,1,0.449219,4,1570,178.512295,358.235071,1.0,36.0,1.0,80.0


In [ ]:
str(fns_trn[0])

In [ ]:
example_dicom = pydicom.dcmread(str(fns_trn[0]))
print(example_dicom)

In [ ]:
# https://www.kaggle.com/jhoward/creating-a-metadata-dataframe

def save_lbls():
    path_lbls = path.join(dir_dcm, 'stage_1_train.csv') 
    lbls = pd.read_csv(path_lbls)
    lbls[["ID","htype"]] = lbls.ID.str.rsplit("_", n=1, expand=True)
    lbls.drop_duplicates(['ID','htype'], inplace=True)
    pvt = lbls.pivot('ID', 'htype', 'Label')
    pvt.reset_index(inplace=True)    
    pvt.to_feather('labels.fth')
    
save_lbls()

In [5]:
df_lbls = pd.read_feather('labels.fth')
df_lbls.head(8)

,ID,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
0,ID_000039fa0,0,0,0,0,0,0
1,ID_00005679d,0,0,0,0,0,0
2,ID_00008ce3c,0,0,0,0,0,0
3,ID_0000950d7,0,0,0,0,0,0
4,ID_0000aee4b,0,0,0,0,0,0
5,ID_0000f1657,0,0,0,0,0,0
6,ID_000178e76,0,0,0,0,0,0
7,ID_00019828f,0,0,0,0,0,0


In [6]:
merge_table = df_lbls.merge(df_trn, left_on='ID', right_on='SOPInstanceUID')
merge_table.head()

,ID,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,SOPInstanceUID,Modality,PatientID,...,MultiPixelSpacing,PixelSpacing1,img_min,img_max,img_mean,img_std,MultiWindowCenter,WindowCenter1,MultiWindowWidth,WindowWidth1
0,ID_000039fa0,0,0,0,0,0,0,ID_000039fa0,CT,ID_eeaf99e7,...,1,0.488281,-2000,2848,13.943810,1195.308120,NaN,NaN,NaN,NaN
1,ID_00005679d,0,0,0,0,0,0,ID_00005679d,CT,ID_18f2d431,...,1,0.460938,-1024,4095,443.519691,903.128368,NaN,NaN,NaN,NaN
2,ID_00008ce3c,0,0,0,0,0,0,ID_00008ce3c,CT,ID_ce8a3cd2,...,1,0.488281,0,2602,491.003143,560.641834,1.0,40.0,1.0,80.0
3,ID_0000950d7,0,0,0,0,0,0,ID_0000950d7,CT,ID_d278c67b,...,1,0.494863,0,1433,60.012962,150.003528,NaN,NaN,NaN,NaN
4,ID_0000aee4b,0,0,0,0,0,0,ID_0000aee4b,CT,ID_ce5f0b6c,...,1,0.423828,10,2368,475.059891,506.609030,1.0,36.0,1.0,80.0


In [7]:
(merge_table.ID == merge_table.SOPInstanceUID).value_counts()

True    674258
dtype: int64

In [8]:
# Some files didn't contain legitimate images, so we need to remove them

png = glob.glob(os.path.join(dir_train_img, '*.png'))
png = [os.path.basename(png)[:-4] for png in png]
png = np.array(png)

# train = train[train['ID'].isin(png)]


In [9]:
(merge_table['ID'].isin(png)).value_counts()

True     674257
False         1
Name: ID, dtype: int64

In [10]:
merge_table = merge_table[merge_table['ID'].isin(png)]
merge_table['Image'] = merge_table.ID

In [11]:
merge_table.head()

,ID,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,SOPInstanceUID,Modality,PatientID,...,PixelSpacing1,img_min,img_max,img_mean,img_std,MultiWindowCenter,WindowCenter1,MultiWindowWidth,WindowWidth1,Image
0,ID_000039fa0,0,0,0,0,0,0,ID_000039fa0,CT,ID_eeaf99e7,...,0.488281,-2000,2848,13.943810,1195.308120,NaN,NaN,NaN,NaN,ID_000039fa0
1,ID_00005679d,0,0,0,0,0,0,ID_00005679d,CT,ID_18f2d431,...,0.460938,-1024,4095,443.519691,903.128368,NaN,NaN,NaN,NaN,ID_00005679d
2,ID_00008ce3c,0,0,0,0,0,0,ID_00008ce3c,CT,ID_ce8a3cd2,...,0.488281,0,2602,491.003143,560.641834,1.0,40.0,1.0,80.0,ID_00008ce3c
3,ID_0000950d7,0,0,0,0,0,0,ID_0000950d7,CT,ID_d278c67b,...,0.494863,0,1433,60.012962,150.003528,NaN,NaN,NaN,NaN,ID_0000950d7
4,ID_0000aee4b,0,0,0,0,0,0,ID_0000aee4b,CT,ID_ce5f0b6c,...,0.423828,10,2368,475.059891,506.609030,1.0,36.0,1.0,80.0,ID_0000aee4b


In [13]:
save_columns = ['Image', 'PatientID', 'epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural', 'any']

trim_merge_table = merge_table[save_columns]
trim_merge_table.head()

,Image,PatientID,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any
0,ID_000039fa0,ID_eeaf99e7,0,0,0,0,0,0
1,ID_00005679d,ID_18f2d431,0,0,0,0,0,0
2,ID_00008ce3c,ID_ce8a3cd2,0,0,0,0,0,0
3,ID_0000950d7,ID_d278c67b,0,0,0,0,0,0
4,ID_0000aee4b,ID_ce5f0b6c,0,0,0,0,0,0


In [15]:
n_split = 6
random_st = 42



# gkf = GroupKFold(n_splits=n_split, shuffle=True, random_state=random_st)
gkf = GroupKFold(n_splits=n_split)

for fold, (train_index, val_index) in enumerate(gkf.split(trim_merge_table, groups=trim_merge_table.PatientID)):
    train_df = trim_merge_table.iloc[train_index]
    val_df = trim_merge_table.iloc[val_index]
    
    print(f'=> Fold {fold} \n---train---')
    print(train_df.loc[:, ['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural', 'any']].sum())
    print('----val----')
    print(val_df.loc[:, ['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural', 'any']].sum())
    
    train_df.to_csv(f'{saved}/train_fold{fold}.csv', index=False)
    val_df.to_csv(f'{saved}/val_fold{fold}.csv', index=False)

=> Fold 0 
---train---
epidural             2317
intraparenchymal    27242
intraventricular    19805
subarachnoid        26986
subdural            35486
any                 80969
dtype: int64
----val----
epidural              444
intraparenchymal     5322
intraventricular     3961
subarachnoid         5136
subdural             7010
any                 16134
dtype: int64
=> Fold 1 
---train---
epidural             2394
intraparenchymal    26852
intraventricular    19816
subarachnoid        26465
subdural            35963
any                 81074
dtype: int64
----val----
epidural              367
intraparenchymal     5712
intraventricular     3950
subarachnoid         5657
subdural             6533
any                 16029
dtype: int64
=> Fold 2 
---train---
epidural             2395
intraparenchymal    26950
intraventricular    19592
subarachnoid        26970
subdural            35678
any                 81189
dtype: int64
----val----
epidural              366
intraparenchymal     561

In [ ]:
train_df.head()

In [ ]:
# Also prepare the test data

test[['ID','Image','Diagnosis']] = test['ID'].str.split('_', expand=True)
test['Image'] = 'ID_' + test['Image']
test = test[['Image', 'Label']]
test.drop_duplicates(inplace=True)

test.to_csv(f'{saved}/test.csv', index=False)

In [ ]:
test.head()